<a href="https://colab.research.google.com/github/masdesouza/FDIA-PdM/blob/master/notebooks/regression_SpeedingUpCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import modules


In [1]:
# Regression models: How many more cycles an in-service engine will last before it fails?
import tensorflow.keras 
from tensorflow.keras.layers import Conv1D, BatchNormalization, Dropout, Dense, InputLayer, Flatten, MaxPool1D, Activation, GlobalAveragePooling1D, Conv2D,Conv3D,Lambda
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential,load_model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

#2. Configuration

In [2]:
# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

# define path to save model
model_path = '/content/drive/MyDrive/PFC/FDIA-PdM/Trained models/CNN/regression_model_CNN.h5'


##2.1  Import dataset

In [3]:
import os
import pickle
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive")

Mounted at /content/drive


#3. Data Ingestion

In [4]:
# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv('/content/drive/MyDrive/PFC/FDIA-PdM/Datasets/CMAPSSData/train_FD002.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

train_df = train_df.sort_values(['id','cycle'])

# read test data - It is the aircraft engine operating data without failure events recorded.
test_df = pd.read_csv('//content/drive/MyDrive/PFC/FDIA-PdM/Datasets/CMAPSSData/test_FD002.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

# read ground truth data - It contains the information of true remaining cycles for each engine in the testing data.
truth_df = pd.read_csv('/content/drive/MyDrive/PFC/FDIA-PdM/Datasets/CMAPSSData/RUL_FD002.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

#4. Data Preprocessing

## 4.1 Train

In [5]:
# Data Labeling - generate column RUL(Remaining Usefull Life or Time to Failure)
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)

# generate label columns for training data
# we will only make use of "label1" for binary classification, 
# while trying to answer the question: is a specific engine going to fail within w1 cycles?
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2

# MinMax normalization (from 0 to 1)
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)

#train_df.to_csv('../../Dataset/PredictiveTraining.csv', encoding='utf-8',index = None)


## 4.2 Test

In [6]:
# MinMax normalization (from 0 to 1)
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
print(test_df.head())

# We use the ground truth dataset to generate labels for the test data.
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)

# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2

#test_df.to_csv('../../Dataset/PredictiveManteinanceTest.csv', encoding='utf-8',index = None)

# pick a large window size of 50 cycles
sequence_length = 100

# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):

    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]

    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

# pick the feature columns
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

# TODO for debug 
# val is a list of 192 - 50 = 142 bi-dimensional array (50 rows x 25 columns)
val=list(gen_sequence(train_df[train_df['id']==1], sequence_length, sequence_cols))
print(len(val))

# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

# function to generate labels
def gen_labels(id_df, seq_length, label):

    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]

    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

   id  cycle  setting1  setting2  ...  s19       s20       s21  cycle_norm
0   1      1  0.238019  0.297150  ...  1.0  0.625514  0.633951    0.000000
1   1      2  0.476162  0.831354  ...  1.0  0.483882  0.500205    0.002653
2   1      3  0.833282  0.997625  ...  1.0  0.164609  0.165078    0.005305
3   1      4  0.999967  0.998812  ...  1.0  0.005830  0.023186    0.007958
4   1      5  0.595089  0.737886  ...  0.0  0.141632  0.145822    0.010610

[5 rows x 27 columns]
49
(27759, 100, 25)


(27759, 1)

# 5. Modeling

In [7]:
ExpandDimension = lambda axis: Lambda(lambda x: K.expand_dims(x, axis))
SqueezeDimension = lambda axis: Lambda(lambda x: K.squeeze(x, axis))

# Layers
def simple_factorized_conv(filters, kernel, *args, **kwargs):
    kwargs["activation"] = None
    def __inner(inp):
        cnn1 = Conv2D(filters, (kernel[0], 1), *args, **kwargs)(inp)
        cnn2 = Conv2D(filters, (1, kernel[1]), *args, **kwargs)(cnn1)
        
        return cnn2

    return __inner
  
def cp_decomposed_conv(filters, kernel, *args, **kwargs):
    """
    Beware, it doesn't work! It's just an aproximate demostration of how
    CP decomposition should be implemented. The issue is in the dimension matching.
    If you found the solution, feel free to comment it, either in the blogpost or here.
    You'll get full credit for your finding.
    """
    kwargs["activation"] = None
    rank = filters // 2
    d = kernel[0]
    def __inner(inp):
        first    = Conv2D(rank, kernel_size=(1, 1), **kwargs)(inp)
        
        expanded = ExpandDimension(axis=1)(first)
        mid1     = Conv3D(rank, kernel_size=(d, 1, 1), **kwargs)(expanded)
        mid2     = Conv3D(rank, kernel_size=(1, d, 1), **kwargs)(mid1)
        squeezed = SqueezeDimension(axis=1)(mid2)
        
        last     = Conv2D(filters,  kernel_size=(1, 1), **kwargs)(squeezed)
        
        return last

    return __inner

def bn_relu(layer):
    def __inner(*args, **kwargs):
        l = layer(*args, **kwargs)
        bn = BatchNormalization()(l)
        act = Activation("relu")(bn)
        
        return act # courtesy to Jiun-Kuei Jung

    return __inner

In [8]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Dropout, Conv2D, SeparableConv2D,
    MaxPool2D, Flatten, GlobalAveragePooling2D
)

class BaseExampleModel:
    def __init__(self, conv_type="std"):
        """
        :convtype: used as a flag to easily switch the Convolution layer implementation
        """
        self.conv_layer = lambda *args, **kwargs: {
                              "std": bn_relu(Conv2D(*args, **kwargs)),
                              "sep": bn_relu(SeparableConv2D(*args, **kwargs)),
                              "cp" : bn_relu(cp_decomposed_conv(*args, **kwargs)),
                              "fac": bn_relu(simple_factorized_conv(*args, **kwargs))
                          }[conv_type]
        
    def make(self, inp, nb_classes):
        """
        :inp: a reference to the Keras Input layer, to easily switch the input size
              and even the way data is fed into network
              (via standard Keras methods or via TF Dataset API)
        :nb_classes: for the final Dense layer
        """
        NotImplemented

class AllCNNLike(BaseExampleModel):
    """
    isnpired from https://arxiv.org/abs/1412.6806
    namely All-CNN-C
    """
    def __init__(self, conv_type="std"):
        super().__init__(conv_type=conv_type)
        
    def make(self, inp, nb_classes):
        conv1 = self.conv_layer(96, (3, 3), padding="same")(inp)
        conv2 = self.conv_layer(96, (3, 3), padding="same")(conv1)
        
        conv3 = bn_relu(Conv2D(96, (3, 3), padding="same", strides=(2, 2)))(conv2)
        
        conv4 = self.conv_layer(192, (3, 3), padding="same")(conv3)
        conv5 = self.conv_layer(192, (3, 3), padding="same")(conv4)
        
        conv6 = bn_relu(Conv2D(192, (3, 3), padding="same", strides=(2, 2)))(conv5)
        
        conv7 = bn_relu(Conv2D(192, (3, 3), padding="same"))(conv6)
        conv8 = bn_relu(Conv2D(192, (1, 1), padding="same"))(conv7)
        conv9 = bn_relu(Conv2D(nb_classes, (1, 1), padding="same"))(conv8)
        
        gap = GlobalAveragePooling2D()(conv9)
        final = Activation("softmax")(gap)
        
        return Model(inputs=inp, outputs=final)
      
class WiderAllCNNLike(BaseExampleModel):
    """
    isnpired from https://arxiv.org/abs/1412.6806
    namely All-CNN-C
    """
    def __init__(self, conv_type="std"):
        super().__init__(conv_type=conv_type)
        
    def make(self, inp, nb_classes):
        conv1 = self.conv_layer(192, (3, 3), padding="same")(inp)
        
        conv2 = bn_relu(Conv2D(96, (3, 3), padding="same", strides=(2, 2)))(conv1)
        
        conv3 = self.conv_layer(384, (3, 3), padding="same")(conv2)
        
        conv4 = bn_relu(Conv2D(128, (3, 3), padding="same", strides=(2, 2)))(conv3)
        
        conv5 = bn_relu(Conv2D(256, (3, 3), padding="same"))(conv4)
        conv6 = bn_relu(Conv2D(nb_classes, (1, 1), padding="same"))(conv5)
        
        gap = GlobalAveragePooling2D()(conv6)
        final = Activation("softmax")(gap)
        
        return Model(inputs=inp, outputs=final)

In [10]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from tensorflow.keras.optimizers import  SGD
from tensorflow.keras.layers import Input

def main(conv_type="std", verbose=False, net_type="simple"):
    inp = Input(shape=(32, 32, 3))
    net = {
        "simple": AllCNNLike(conv_type=conv_type).make(inp, nb_classes=100),
        "wide": WiderAllCNNLike(conv_type=conv_type).make(inp, nb_classes=100)
    }[net_type]

    net.compile(SGD(lr=0.01, decay=0.001, momentum=0.8), "categorical_crossentropy",
                 metrics=[categorical_accuracy, top_k_categorical_accuracy])

    if verbose:
        print(net.summary())

    net.fit_generator(train_df, steps_per_epoch=50000 // BATCH_SIZE, epochs=5, verbose=verbose)
    loss, acc, topk_acc = net.evaluate_generator(test_df, steps=10000 // BATCH_SIZE)

    print(f"Accuracy: {acc}, Top5 Accuracy {topk_acc} and Loss {loss}.")

In [ ]:







def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
# Next, we build a deep network. 
# The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. 

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

cnn = Sequential()
cnn.add(InputLayer(input_shape=(sequence_length, nb_features)))
cnn.add(BatchNormalization(axis=-1))  #Scaling the data

cnn.add(Conv1D(filters=64,
               kernel_size=3,
               padding="valid",
               activation="relu",
               kernel_regularizer='l2'
               )
       )
# cnn.add(MaxPool1D(pool_size=2))
# cnn.add(BatchNormalization(axis=-1))
cnn.add(Conv1D(filters=64,
               kernel_size=3,
               padding="valid",
               activation="relu",
               kernel_regularizer='l2')
       )

cnn.add(Conv1D(filters=64,
               kernel_size=3,
               padding="valid",
               activation="relu",
               kernel_regularizer='l2')
       )
# cnn.add(BatchNormalization(axis=-1))
# cnn.add(MaxPool1D(pool_size=2))
# cnn.add(BatchNormalization(axis=-1))

cnn.add(Conv1D(filters=64,
               kernel_size=3,
               padding="valid",
               activation="relu",
               kernel_regularizer='l2')
       )

cnn.add(Flatten())
cnn.add(Dense(40))
cnn.add(Activation('relu'))
cnn.add(Dense(30))
cnn.add(Activation('relu'))
cnn.add(Dense(units=nb_out))
cnn.add(Activation("relu"))
# cnn.add(Dense(30))
# cnn.add(Activation('relu'))
# cnn.add(Dense(20))
# cnn.add(Activation('relu'))
# cnn.add(Dense(10))
# cnn.add(Activation('relu'))
# cnn.add(Dense(units=nb_out))
# cnn.add(Activation('relu'))
cnn.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=[rmse,r2_keras])

print(cnn.summary())

history = cnn.fit(seq_array, label_array, epochs=100, batch_size=512, validation_split=0.05, verbose=2,
          callbacks = [tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       tensorflow.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
    )

# model = Sequential()
# model.add(LSTM(
#          input_shape=(sequence_length, nb_features),
#          units=100,
#          return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(
#           units=100,
#           return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(units=nb_out))
# model.add(Activation("linear"))
# model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])
#
# print(model.summary())
#
#
#
# # fit the network
# history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=2,
#           callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
#                        keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
#           )

# list all data in history
print(history.history.keys())


# training metrics
scores = cnn.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('\nMAE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))

y_pred = cnn.predict(seq_array,verbose=1, batch_size=200)
y_true = label_array

test_set = pd.DataFrame(y_pred)
test_set.to_csv('/content/drive/MyDrive/PFC/FDIA-PdM/Output/submit_train_CNN.csv', index = None)

#6. EVALUATE ON TEST DATA

In [ ]:
# We pick the last sequence for each id in the test data
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
print("seq_array_test_last")
#print(seq_array_test_last)
print(seq_array_test_last.shape)

# Similarly, we pick the labels
#print("y_mask")
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]
label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
print(label_array_test_last.shape)
print("label_array_test_last")
print(label_array_test_last)

# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    cnn.load_weights(model_path)
    #estimator = load_model(model_path,custom_objects={'r2_keras': r2_keras})

    # test metrics
    scores_test = cnn.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
    print('\nMAE: {}'.format(scores_test[1]))
    print('\nR^2: {}'.format(scores_test[2]))

    y_pred_test = cnn.predict(seq_array_test_last)
    y_true_test = label_array_test_last
    print("Prediction")
    print(y_pred_test);
    print("Truth")
    print(y_true_test);

    # Plot in blue color the predicted data and in green color the
    # actual data to verify visually the accuracy of the model.
    fig_verify = plt.figure(figsize=(100, 50))
    plt.plot(y_pred_test, color="blue")
    plt.plot(y_true_test, color="green")
    plt.title('prediction')
    plt.ylabel('RUL in cycles')
    plt.xlabel('Engine ID')
    plt.legend(['predicted', 'actual data'], loc='upper left')
    plt.show()
    fig_verify.savefig("/content/drive/MyDrive/PFC/FDIA-PdM/Output/model_regression_verify_CNN.png")